Running DESI software version 20.7

Compare DESI SV redshifts with external redshifts

In [ ]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, hstack, join
import fitsio
# from astropy.io import fits

from desitarget import targets
from util import compare_redshifts

In [ ]:
params = {'legend.fontsize': 'large',
         'axes.labelsize': 'large',
         'axes.titlesize':'large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [ ]:
# Load redrock catalogs from Blanc deep coadds

tileid_list = [80605, 80607, 80609, 80620, 80622]  # December run LRG+QSO tiles
data_dir = '/global/cfs/cdirs/desi/spectro/redux/blanc/tiles'

zbest_all = []

for tileid in tileid_list:
    fn_list = sorted(glob.glob(os.path.join(data_dir, str(tileid), 'deep', 'zbest-*.fits')))
    zbest = []
    for fn in fn_list:
        tmp1 = Table.read(fn, hdu='ZBEST')
        tmp2 = Table.read(fn, hdu='FIBERMAP')
        # remove duplicates in FIBERMAP
        _, idx = np.unique(tmp2['TARGETID'], return_index=True)
        tmp2 = tmp2[idx]
        if not np.all(tmp1['TARGETID']==tmp2['TARGETID']):
            raise ValueError()        
        tmp = join(tmp1, tmp2, keys='TARGETID', metadata_conflicts='silent')
        zbest.append(tmp)
    zbest = vstack(zbest)
    zbest['tileid'] = tileid
    print(tileid, len(zbest))
    
    zbest_all.append(zbest)
    
zbest = vstack(zbest_all)
print(len(zbest), len(np.unique(zbest['TARGETID'])))

In [ ]:
# Remove FIBERSTATUS!=0 fibers
mask = zbest['FIBERSTATUS']==0
# Remove ZWARN!=0 fibers in zbest coadds
mask &= zbest['ZWARN']==0
# Select QSO targets
mask &= (zbest['SV1_DESI_TARGET'] & 2**2>0)

zbest = zbest[mask]
print(len(mask))

In [ ]:
# Load DR16Q catalog
ls = Table(fitsio.read('/global/cfs/cdirs/desi/target/analysis/truth/dr9.0/south/matched/ls-dr9.0-DR16Q_v4-match.fits', columns=['OBJID', 'BRICKID', 'RELEASE']))
truth = Table(fitsio.read('/global/cfs/cdirs/desi/target/analysis/truth/dr9.0/south/matched/DR16Q_v4-match.fits', columns=['Z']))
ls['targetid'] = targets.encode_targetid(ls['OBJID'], ls['BRICKID'], ls['RELEASE'])

# Match to redrock catalog
_, idx_desi, idx_truth = np.intersect1d(zbest['TARGETID'], ls['targetid'], return_indices=True)
print('Found {} matched objects'.format(len(idx_desi)))

In [ ]:
outlier_threshold = 0.01
compare_redshifts(truth['Z'][idx_truth], zbest['Z'][idx_desi],
                markersize=3, alpha=1, zmin=-0.05, zmax=4., outlier_threshold=outlier_threshold, dz_range=0.01,
                xlabel='$z_{\mathrm{SDSS}}$', ylabel='$z_{\mathrm{DESI}}$', ylabel2='$\\Delta z/(1+z_{\\mathrm{SDSS}})$')